In [1]:
import os
import glob
import xarray as xr
import numpy as np
import shutil
import random
import string
from pathlib import Path

from dask import delayed
import dask
from dask.distributed import Client
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status'})

In [2]:
#define regular mesh in multiples of R
reg_x = np.arange(-3.1,3.2,.1)
reg_y = np.arange(-3,3.1,.1)
X,Y = np.meshgrid(reg_x,reg_y)

In [3]:
vars=['a_ice','m_ice','fh','temp_100','temp_0','temp_50','temp_205','salt_100']

In [4]:
@delayed
def do_stuff(file_temp100,file,out_dir):
    t100 = xr.open_dataset(file_temp100)['temp_100'].astype('float32')
    ne = t100.shape[1]
    ds = xr.open_dataset(file).astype('float32')
    ds['warm_index'] = xr.DataArray(np.empty((1,ne), dtype='bool'), dims=['time','ee']) #predefine
    ds['warm_index'][0,:] = (t100[0,:,np.argmin(np.abs(reg_y)),np.argmin(np.abs(reg_x))]>0).astype(bool) #write
    ds.to_netcdf(out_dir+Path(file).name)

In [5]:
def generate_random_string(length):
    # Define the characters to choose from
    characters = string.ascii_letters + string.digits  # You can add more characters if needed
    # Use random.choices to generate a list of random characters
    random_characters = random.choices(characters, k=length)
    # Join the characters into a string
    random_string = ''.join(random_characters)
    return random_string
#spawn a parallel cluster
n_cores = 20
mem_lim = str(int(100*np.floor(960/n_cores)))+'MB' #96GB total memory, set to MB (96000), divide by number of cores, then round to next 100
dask_dir = '/p/scratch/chhb19/mueller29/dask_dir/'+generate_random_string(10)
if os.path.exists(dask_dir):
    shutil.rmtree(dask_dir)
if 'client' in locals() or 'client' in globals():
    client.close()
client = Client(local_directory=dask_dir,n_workers=n_cores, threads_per_worker=1,memory_limit=mem_lim)
client.amm.start()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/vasco.mueller_at_awi.de/xdaf770f02c4451a8e26967580f03894/proxy/8787/status,
Dashboard: /user/vasco.mueller_at_awi.de/xdaf770f02c4451a8e26967580f03894/proxy/8787/status,Workers: 20
Total threads: 20,Total memory: 89.41 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33533,Workers: 20
Dashboard: /user/vasco.mueller_at_awi.de/xdaf770f02c4451a8e26967580f03894/proxy/8787/status,Total threads: 20
Started: Just now,Total memory: 89.41 GiB
Comm: tcp://127.0.0.1:39361,Total threads: 1
Dashboard: /user/vasco.mueller_at_awi.de/xdaf770f02c4451a8e26967580f03894/proxy/45009/status,Memory: 4.47 GiB
Nanny: tcp://127.0.0.1:44157,


In [6]:
vars=['temp_50','temp_205','salt_100']

In [7]:
results = []
for var in vars:
    in_dir = '/p/scratch/chhb19/mueller29/composites/100m/'+var+'/'
    files = in_dir+'composite_'+var+'*.nc'
    files=glob.glob(files)
    files.sort()
    files_temp100 = '/p/scratch/chhb19/mueller29/composites/100m/temp_100/composite_temp_100*.nc'
    files_temp100=glob.glob(files_temp100)
    files_temp100.sort()
    out_dir = in_dir+'new/'
    os.makedirs(out_dir, exist_ok=True)
    for ii in range(len(files)):
        file_temp100=files_temp100[ii]
        file=files[ii]
        results.append(do_stuff(file_temp100,file,out_dir))

In [8]:
result=dask.compute(*results)

In [9]:
client.close()